# Pivot tabulky

__[Animace operací s dataframy](https://github.com/nathancarter/dataframe-animations?tab=readme-ov-file)__

In [78]:
import pandas as pd

In [79]:
pivot_cholesterol = pd.read_csv("https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/pivot-tabulky/pivot-tabulky/pivot_cholesterol.csv")
pivot_cholesterol.head()


,fdc_id,amount,nutrient_name,branded_food_category
0,1107295,0.0,Cholesterol,Chipsy
1,1107475,65.0,Cholesterol,Sýry
2,1108349,0.0,Cholesterol,Cereálie
3,1109393,100.0,Cholesterol,Sýry
4,1109400,36.0,Cholesterol,Sušenky


### Funkce `cut`

Na zakládě nějaké číselné hodnoty můžeme data rozdělit i do skupin. Každá skupina potřebuje dvě věci:

- číselný interval, který udává rozsah pro zařazení do skupiny,
- označení skupiny.

Uvažujme následující skupiny.

| Obsah cholesterolu     | Kategorie           |
|------------------------|---------------------|
| 0-20                   | Nízký cholesterol     |
| 20-100                 | Střední cholesterol|
| 100-inf                | Vysoký cholesterol    |

Každé potravině můžeme přiřadit popisek, která nám usnadní psaní dotazů.

In [80]:
bins = [0, 20, 100, float('inf')]
labels = ['Nízký cholesterol', 'Střední cholesterol', 'Vysoký cholesterol']

pivot_cholesterol['cholesterol_category'] = pd.cut(pivot_cholesterol['amount'], bins=bins, labels=labels)
pivot_cholesterol

,fdc_id,amount,nutrient_name,branded_food_category,cholesterol_category
0,1107295,0.0,Cholesterol,Chipsy,NaN
1,1107475,65.0,Cholesterol,Sýry,Střední cholesterol
2,1108349,0.0,Cholesterol,Cereálie,NaN
3,1109393,100.0,Cholesterol,Sýry,Střední cholesterol
4,1109400,36.0,Cholesterol,Sušenky,Střední cholesterol
...,...,...,...,...,...
2975,2659382,83.0,Cholesterol,Sýry,Střední cholesterol
2976,2660573,60.0,Cholesterol,Sýry,Střední cholesterol
2977,2660613,0.0,Cholesterol,Slané snacky,NaN
2978,2661380,0.0,Cholesterol,Chipsy,NaN


## Pivot tabulka

Pivot tabulku můžeme vytvořit několika způsoby, jednou z nich je funkce `crosstab()`. 

Pokud chceme v tabulce vidět počet potravin v jednotlivých kombinacích sloupců `branded_food_category` a `cholesterol_category`, stačí nám vložit tyto dva sloupce jako série.

In [81]:
pd.crosstab(pivot_cholesterol["branded_food_category"], pivot_cholesterol["cholesterol_category"])


cholesterol_category,Nízký cholesterol,Střední cholesterol,Vysoký cholesterol
branded_food_category,,,
Cereálie,3,0,0
Chipsy,5,3,0
Cukrovinky,60,18,0
Další snacky,6,31,20
En. tyčinky,31,16,1
Nakl. zelenina,15,11,1
Pečivo,8,23,0
Slané snacky,41,6,0
Sušenky,36,75,3


Pro zobrazení součtu řádků a sloupců můžeme využít parametr `margins`.


In [82]:
pd.crosstab(pivot_cholesterol["branded_food_category"], pivot_cholesterol["cholesterol_category"], margins=True)


cholesterol_category,Nízký cholesterol,Střední cholesterol,Vysoký cholesterol,All
branded_food_category,,,,
Cereálie,3,0,0,3
Chipsy,5,3,0,8
Cukrovinky,60,18,0,78
Další snacky,6,31,20,57
En. tyčinky,31,16,1,48
Nakl. zelenina,15,11,1,27
Pečivo,8,23,0,31
Slané snacky,41,6,0,47
Sušenky,36,75,3,114


### Parameter `normalize`

Pokud bychom chtěli hodnoty převést na procenta, můžeme využít parametr `normalize`. 

Jak normalize funguje:

- `normalize="index"`: Normalizuje tabulku po řádcích. Součet hodnot v každém řádku bude 1 (nebo 100 %). Každá buňka tedy představuje relativní podíl hodnoty v rámci daného řádku.
- `normalize="columns"`: Normalizuje tabulku po sloupcích. Součet hodnot v každém sloupci bude 1 (nebo 100 %). Každá buňka představuje relativní podíl hodnoty v rámci daného sloupce.
- `normalize=True`: Normalizuje tabulku podle všech hodnot dohromady. Celkový součet všech hodnot v tabulce bude 1 (nebo 100 %). Každá buňka tedy představuje relativní podíl své hodnoty v celé tabulce.
- Bez parametru `normalize` (implicitní hodnota): Tabulka obsahuje absolutní četnosti (nebo hodnoty v případě agregace).

In [83]:
pd.crosstab(pivot_cholesterol["branded_food_category"], 
            pivot_cholesterol["cholesterol_category"], 
            normalize="index"       
)#.style.format("{:.0%}")



cholesterol_category,Nízký cholesterol,Střední cholesterol,Vysoký cholesterol
branded_food_category,,,
Cereálie,1.000000,0.000000,0.000000
Chipsy,0.625000,0.375000,0.000000
Cukrovinky,0.769231,0.230769,0.000000
Další snacky,0.105263,0.543860,0.350877
En. tyčinky,0.645833,0.333333,0.020833
Nakl. zelenina,0.555556,0.407407,0.037037
Pečivo,0.258065,0.741935,0.000000
Slané snacky,0.872340,0.127660,0.000000
Sušenky,0.315789,0.657895,0.026316


## Agragace & pivot tabulka

In [84]:
pivot_cholesterol_fiber = pd.read_csv("https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/pivot-tabulky/pivot-tabulky/pivot_cholesterol_fiber.csv")

pivot_cholesterol_fiber.head()


,fdc_id,amount,nutrient_name,branded_food_category
0,1107295,0.0,Cholesterol,Chipsy
1,1107475,65.0,Cholesterol,Sýry
2,1108349,0.0,Cholesterol,Cereálie
3,1109393,100.0,Cholesterol,Sýry
4,1109400,36.0,Cholesterol,Sušenky


In [85]:
pd.crosstab(pivot_cholesterol_fiber["branded_food_category"], 
            pivot_cholesterol_fiber["nutrient_name"], 
            pivot_cholesterol_fiber["amount"], 
            aggfunc="mean")

nutrient_name,Cholesterol,"Fiber, total dietary"
branded_food_category,,
Cereálie,0.241176,7.539344
Chipsy,0.948598,5.423182
Cukrovinky,5.498305,1.382993
Další snacky,34.854015,3.278571
En. tyčinky,6.450000,8.783422
Nakl. zelenina,6.708661,1.890244
Pečivo,5.305085,3.735246
Slané snacky,1.526096,7.230321
Sušenky,18.731061,2.280075


Další funce pro vytváření pivot tabulek v **pandas**:

| Funkce       | Popis                                                                                                    | Agregace dat | Na jaké vstupy je určena    | Další poznámky                                                                                                                                                  |
|--------------|----------------------------------------------------------------------------------------------------------|--------------|-----------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `pivot`      | Vytváří jednoduchou kontingenční tabulku na základě jedinečných hodnot zadaných sloupců.                  | **Ne**       | Pro jedinečné kombinace      | Nefunguje, pokud existují duplicity v kombinaci sloupců (`index`, `columns`). V takovém případě vyvolá chybu.                                                    |
| `pivot_table`| Rozšířená verze `pivot`, umožňuje agregaci dat pomocí funkce (např. `mean`, `sum`, `count`).              | **Ano**      | Pro numerická data i duplicitní kombinace | Umožňuje použít více agregačních funkcí, má více možností pro práci s duplicitami ve sloupcích.                                                              |
| `crosstab`   | Vytváří křížovou tabulku mezi dvěma (nebo více) kategoriemi, a to na základě četností nebo jiné agregace. | **Ano**      | Pro kategorická data         | Nejčastěji používána pro četnostní počítání (defaultně `count`), ale může provádět i jiné operace na numerických datech.                                        |

---
## Cvičení

### Booking

Stáhni si data ze souboru o rezervacích hotelů ze serveru Booking.com. Data jsou uložená v souboru [hotel_bookings.csv](https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/pivot-tabulky/dalsi-funkce/booking/hotel_bookings.csv). U rezervací evidujeme, jestli byly zrušené, to najdeme ve sloupci `is_canceled` (1 pro zrušené rezervace a 0 pro nezrušené). Vytvoř kontingenční tabulku, která porovná počet zrušených rezervací podle typu hotelu (sloupec `hotel`). Je více rezervací zrušeno pro městské hotely nebo pro hotely v rezortech?

Dále zkus rezervace rozdělit do skupin podle toho, v jakém předstihu byly rezervace provedeny. Zaměř se pouze na rezervace v městkých hotelech, tj. vytvoř tabulku, která bude obsahovat pouze data, které mají ve sloupci `hotel` hodnotu `City Hotel`- Využij sloupec `lead_time`. Níže máš skupiny, podle kterých můžeš data rozdělit. Vytvoř si pivot tabulku, která zobrazuje počty rezervací v jednotlivých kategoriích v závislosti na tom, jestli byly zrušeny nebo ne. Pro které kategorie je více zrušených rezervací a pro které naopak více nezrušených? A v jaké skupině je celkově nejvíce rezervací?

| Lead Time              | Reservation Category    |
|------------------------|-------------------------|
| 0-7                    | Last-minute             |
| 8-30                   | Short-term              |
| 31-180                 | Medium-term             |
| 180-inf                | Long-term               |


In [86]:
data = pd.read_csv(
    "https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/pivot-tabulky/pivot-tabulky/booking/hotel_bookings.csv"
)

In [87]:
pd.crosstab(data["hotel"],
            data["is_canceled"],
            normalize="index"
            ).style.format("{:.0%}")

is_canceled,0,1
hotel,,
City Hotel,58%,42%
Resort Hotel,72%,28%


In [88]:
data = data[data["hotel"] == "City Hotel"]

bins = [0, 7, 30, 180, float("inf")]
labels = ["Last-minute", "Short-term", "Medium-term", "Long-term"]
data["lead_time_category"] = pd.cut(data["lead_time"], bins=bins, labels=labels)

data_pivot = pd.crosstab(data["lead_time_category"], data["is_canceled"])
data_pivot.rename(columns={0: "Canceled", 1: "Ok"}, inplace=True)
data_pivot

is_canceled,Canceled,Ok
lead_time_category,,
Last-minute,6653,1046
Short-term,8674,3880
Medium-term,21974,17046
Long-term,6091,10857


### Titanic a další pivot tabulky

K tomuto cvičení využij data o cestujících na Titanicu ze souboru [titanic.csv](https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/pivot-tabulky/dalsi-funkce/titanic/titanic.csv).

Vytvoř další kontingenční tabulku, která porovná závislost mezi třídou (sloupec `Pclass`), ve které cestoval, a tím, jestli přežil. Zkus spočítat počty přeživších z každé třídy. Dále zkus vypočítat relativní počet přeživších pro jednotlivé třídy. Dále zkus vytvořit tabulku s relativním počtem přeživších v závislosti na pohlaví (sloupec `Sex`).

In [89]:
data = pd.read_csv(
    "https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/pivot-tabulky/pivot-tabulky/titanic/titanic.csv"
)
data_pivot = pd.crosstab(data["Pclass"], data["Survived"])
data_pivot

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


In [90]:
data_pivot = pd.crosstab(data["Pclass"], data["Survived"], normalize="index")
data_pivot.style.format("{:.2%}")

Survived,0,1
Pclass,,
1,37.04%,62.96%
2,52.72%,47.28%
3,75.76%,24.24%


In [91]:
data_pivot = pd.crosstab(data["Sex"], data["Survived"])
data_pivot

Survived,0,1
Sex,,
female,81,233
male,468,109


In [92]:
data_pivot = pd.crosstab(data["Sex"], data["Survived"], normalize="index")
data_pivot.style.format("{:.0%}")

Survived,0,1
Sex,,
female,26%,74%
male,81%,19%
